In [1]:
# 📌 Importar librerías
import pandas as pd
import numpy as np

#  Ruta de los archivos CSV en la carpeta "raw"
raw_path = "C:/Users/pmate/OneDrive - Universidad San Francisco de Quito/8 SEMESTRE/DataMining/Examanes/Proyecto_Megaline/data/raw/"

#  Cargar los datasets
df_users = pd.read_csv(raw_path + "megaline_users.csv")
df_calls = pd.read_csv(raw_path + "megaline_calls.csv")
df_sms = pd.read_csv(raw_path + "megaline_messages.csv")
df_internet = pd.read_csv(raw_path + "megaline_internet.csv")
df_plans = pd.read_csv(raw_path + "megaline_plans.csv")

#  **1️⃣ Manejo de valores faltantes**
# Los únicos valores nulos están en churn_date (usuarios que no han cancelado su plan). No es un problema.
df_users["churn_date"].fillna("Activo", inplace=True)

#  **2️⃣ Tratamiento de outliers**
# 📌 Llamadas: Eliminar llamadas con duración = 0 (posibles errores o llamadas fallidas)
df_calls = df_calls[df_calls["duration"] > 0]

# SMS: No hay datos faltantes, pero revisamos outliers
sms_counts = df_sms.groupby("user_id").size()
sms_outlier_threshold = sms_counts.quantile(0.99)  # Top 1% como outliers
df_sms = df_sms[df_sms["user_id"].map(sms_counts) <= sms_outlier_threshold]

#  Internet: Identificar y manejar outliers
data_usage = df_internet.groupby("user_id")["mb_used"].sum()
data_outlier_threshold = data_usage.quantile(0.99)  # Top 1% como outliers
df_internet = df_internet[df_internet["user_id"].map(data_usage) <= data_outlier_threshold]

#  **3️⃣ Unificación de las tablas**
# 🔹 Agrupar consumo por usuario
df_calls_grouped = df_calls.groupby("user_id")["duration"].sum().reset_index()
df_calls_grouped.rename(columns={"duration": "total_minutos"}, inplace=True)

df_sms_grouped = df_sms.groupby("user_id").size().reset_index(name="total_sms")

df_internet_grouped = df_internet.groupby("user_id")["mb_used"].sum().reset_index()
df_internet_grouped.rename(columns={"mb_used": "total_mb"}, inplace=True)

# 🔹 Unimos los datos de consumo con los usuarios y planes
df_users = df_users.merge(df_plans, left_on="plan", right_on="plan_name", how="left")
df_users = df_users.merge(df_calls_grouped, on="user_id", how="left").fillna(0)
df_users = df_users.merge(df_sms_grouped, on="user_id", how="left").fillna(0)
df_users = df_users.merge(df_internet_grouped, on="user_id", how="left").fillna(0)

#  **4️⃣ Cálculo de Excesos y Costos Extra**
df_users["exceso_minutos"] = (df_users["total_minutos"] - df_users["minutes_included"]).clip(lower=0)
df_users["exceso_sms"] = (df_users["total_sms"] - df_users["messages_included"]).clip(lower=0)
df_users["exceso_mb"] = (df_users["total_mb"] - df_users["mb_per_month_included"]).clip(lower=0)

#  Cálculo del costo extra por sobreconsumo
df_users["costo_extra_minutos"] = df_users["exceso_minutos"] * df_users["usd_per_minute"]
df_users["costo_extra_sms"] = df_users["exceso_sms"] * df_users["usd_per_message"]
df_users["costo_extra_mb"] = (df_users["exceso_mb"] / 1024) * df_users["usd_per_gb"]

#  Cálculo del costo total que paga cada usuario
df_users["costo_total"] = df_users["usd_monthly_pay"] + df_users["costo_extra_minutos"] + df_users["costo_extra_sms"] + df_users["costo_extra_mb"]

#  **5️⃣ Guardamos el dataset limpio**
clean_path = "C:/Users/pmate/OneDrive - Universidad San Francisco de Quito/8 SEMESTRE/DataMining/Examanes/Proyecto_Megaline/data/clean/"
df_users.to_csv(clean_path + "usuarios_consumo_limpio.csv", index=False)

print("\n Data Wrangling completado. Archivo guardado: usuarios_consumo_limpio.csv")



 Data Wrangling completado. Archivo guardado: usuarios_consumo_limpio.csv


In [ ]:
# Se realizó Data Wrangling para limpiar y unificar los datos. Primero, se manejaron valores faltantes, 
# reemplazando los nulos en churn_date por "Activo". Luego, se eliminaron outliers: llamadas de 0 minutos, 
# usuarios con SMS y consumo de datos en el top 1% fueron filtrados. Se agruparon los datos de consumo de 
# llamadas, SMS y MB por usuario, asegurando que los JOINs no eliminaran información importante. Se calcularon 
# los excesos de minutos, SMS y datos en relación con los límites del plan y se estimaron los costos extra por 
# sobreconsumo. Finalmente, se sumaron estos costos al valor del plan base para obtener el costo total que 
# cada usuario pagará. El dataset final fue guardado como "usuarios_consumo_limpio.csv" y está listo para 
# Feature Engineering y Modelado.
